# Trend Detection

In [2]:
import warnings
warnings.filterwarnings('ignore')

import time
from datetime import date, datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pandas_ta as ta
from ps_candlestick import get_type_candlestick
from ps_candlestick import has_reversal_pattern

In [3]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)
dataset

,Open,High,Low,Close,Volume
Date,,,,,
2018-08-13 09:00:00,943.5,943.6,942.9,943.1,1812
2018-08-13 09:05:00,943.1,943.5,942.9,943.3,1323
2018-08-13 09:10:00,943.2,943.3,942.6,943.1,1207
2018-08-13 09:15:00,943.1,943.1,942.3,942.6,1196
2018-08-13 09:20:00,942.6,943.7,942.4,943.7,1765
...,...,...,...,...,...
2024-07-10 14:15:00,1314.6,1315.9,1312.1,1314.3,9556
2024-07-10 14:20:00,1314.2,1314.4,1310.3,1310.7,10901
2024-07-10 14:25:00,1310.7,1312.9,1310.0,1310.3,8964


In [4]:
data2 = dataset.copy()
data2["ema_line"] = ta.ema(data2["Close"], length=20)
data2['above_ma'] = data2.apply(lambda r: 1 if r['Close'] > r['ema_line'] else 0, axis=1)
data2['below_ma'] = data2.apply(lambda r: 1 if r['Close'] < r['ema_line'] else 0, axis=1)
data2['total_above_ma'] = data2['above_ma'].rolling(150).sum()
data2['total_below_ma'] = data2['below_ma'].rolling(150).sum()
data2['trend'] = data2.apply(lambda r: 'switch' if r['total_above_ma'] == r['total_below_ma'] else ('up' if r['total_above_ma'] > r['total_below_ma'] else 'down'), axis=1)
data2['trend_shift'] = data2['trend'].shift(1)

In [5]:
data2

,Open,High,Low,Close,Volume,ema_line,above_ma,below_ma,total_above_ma,total_below_ma,trend,trend_shift
Date,,,,,,,,,,,,
2018-08-13 09:00:00,943.5,943.6,942.9,943.1,1812,NaN,0,0,NaN,NaN,down,None
2018-08-13 09:05:00,943.1,943.5,942.9,943.3,1323,NaN,0,0,NaN,NaN,down,down
2018-08-13 09:10:00,943.2,943.3,942.6,943.1,1207,NaN,0,0,NaN,NaN,down,down
2018-08-13 09:15:00,943.1,943.1,942.3,942.6,1196,NaN,0,0,NaN,NaN,down,down
2018-08-13 09:20:00,942.6,943.7,942.4,943.7,1765,NaN,0,0,NaN,NaN,down,down
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-10 14:15:00,1314.6,1315.9,1312.1,1314.3,9556,1316.252254,0,1,77.0,73.0,up,up
2024-07-10 14:20:00,1314.2,1314.4,1310.3,1310.7,10901,1315.723468,0,1,76.0,74.0,up,up
2024-07-10 14:25:00,1310.7,1312.9,1310.0,1310.3,8964,1315.206947,0,1,75.0,75.0,switch,up


In [6]:
data2[data2.trend=='switch']

,Open,High,Low,Close,Volume,ema_line,above_ma,below_ma,total_above_ma,total_below_ma,trend,trend_shift
Date,,,,,,,,,,,,
2018-08-16 13:20:00,937.4,938.4,937.3,938.2,850,938.909621,0,1,75.0,75.0,switch,up
2018-08-17 10:25:00,953.0,953.3,952.3,952.5,1130,950.922888,1,0,75.0,75.0,switch,down
2018-08-17 13:30:00,952.1,952.1,950.3,950.3,2899,952.027704,0,1,75.0,75.0,switch,up
2018-08-21 10:45:00,946.6,946.8,946.4,946.5,979,946.193025,1,0,75.0,75.0,switch,down
2018-08-30 10:05:00,961.4,962.1,961.3,962.0,715,962.184906,0,1,75.0,75.0,switch,up
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-02 10:45:00,1291.3,1291.8,1290.4,1291.2,3442,1287.861400,1,0,75.0,75.0,switch,down
2024-07-02 10:50:00,1291.2,1292.3,1291.2,1291.5,5420,1288.207933,1,0,75.0,75.0,switch,switch
2024-07-09 10:00:00,1306.3,1306.3,1305.1,1305.9,3134,1309.133707,0,1,75.0,75.0,switch,up


In [9]:
_1130_data = data2[100*data2.index.hour + data2.index.minute == 1130]
_1130_data

,Open,High,Low,Close,Volume,ema_line,above_ma,below_ma,total_above_ma,total_below_ma,trend,trend_shift
Date,,,,,,,,,,,,
2018-08-13 11:30:00,947.1,947.3,947.1,947.3,51,945.019328,1,0,NaN,NaN,down,down
2018-08-14 11:30:00,954.3,954.3,954.3,954.3,16,954.114180,1,0,NaN,NaN,down,down
2018-08-15 11:30:00,958.1,958.1,958.0,958.0,66,958.870229,0,1,NaN,NaN,down,down
2018-08-16 11:30:00,938.1,938.1,938.1,938.1,11,939.610455,0,1,80.0,70.0,up,up
2018-08-17 11:30:00,954.3,954.3,954.3,954.3,8,951.785498,1,0,78.0,72.0,up,up
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-04 11:30:00,1309.0,1309.0,1309.0,1309.0,92,1308.709883,1,0,131.0,19.0,up,up
2024-07-05 11:30:00,1309.0,1309.0,1308.7,1308.7,72,1309.812869,0,1,109.0,41.0,up,up
2024-07-08 11:30:00,1308.4,1308.4,1308.3,1308.3,16,1309.863328,0,1,93.0,57.0,up,up
